# EP Tender : simulation par chaînes de Markov

## Description du modèle

On considère une station comportant $f$ files, pour un total de $n$ tenders en circulation. Chaque tender comporte $d + 1$ états de charge, numérotés de 0 à $d$. À 0 un tender est complètement déchargé, et à $d$ il est complètement chargé.

On utilise un modèle à temps discret.

Les voitures clientes arrivent de sorte que l'écart temporel entre la $i$-ième voiture et la $(i+1)$-ième est de loi géométrique de paramètre $\lambda$. Lorsqu'elles arrivent elles doivent prendre un tender complètement chargé qu'elles ramènent complètement déchargé au début de l'étape suivante.

Fatalement, un jour plus de $n$ voitures arriveront en même temps et alors la demande ne pourra pas être satisfaite. On note $T_\mathrm{bug}$ le premier temps où la demande en Tender est insatisfaite et on s'intéresse à son espérance. On cherche à trouver, pour un processus de rangement et de sélection des tenders donné, l'espérance de $T_\mathrm{bug}$, qu'on souhaite maximal.

Chaque étape temporelle se déroule comme suit :
1. Tous les tenders dans la station gagnent un niveau de charge, pour un niveau maximum de $d$.
2. Les tenders partis à l'étape précédente sont rangés dans la station.
3. Les nouvelles voitures clientes arrivent et des tenders chargés leur sont attribués.
4. Si la demande a pu être satisfaite, on passe au temps suivant.

## Importation des modules

In [1]:
import numpy as np

## Encodage des états

### Encodage d'une file simple

Une file donnée contient un certain nombre de tenders, chacun à un état de charge donné. Remarquons que les tenders les plus au fond sont ceux arrivés le plus tôt, ils sont donc les plus chargés. On peut donc représenter une file par une suite décroissante d'entiers entre 0 et $d$, de longueur au plus $n$. Si $d < n$ il peut être plus efficace d'encoder cette suite par une suite de $d + 1$ entiers positifs, de somme au plus $n$, représentant le nombre de tenders de niveau de charge $d$ dans la file, puis le nombre de tenders de niveau de charge $d - 1$, et ainsi de suite jusque $0$.

In [2]:
def encode(file, d) :
# Prend une suite décroissante d'entiers positifs entre 0 et d, et renvoie le nombre d'entiers pour chaque valeur

    fileEncodee = np.zeros(d + 1, dtype = int)
    for charge in file :
        fileEncodee[d - charge] += 1
    
    return fileEncodee

In [3]:
file = [5, 5, 3, 2, 2, 1, 0]
fileEncodee = encode(file, 6)
fileEncodee

array([0, 2, 0, 1, 2, 1, 1])

In [4]:
def decode(fileEncodee) :
    # Prend une suite d'entiers positifs, et renvoie une suite d'entiers, de longueur la somme de la suite originale
    # et prenant chaque valeur le nombre de fois indiqué par la suite de départ, de sorte à se terminer par la valeur 0
    
    d = len(fileEncodee) - 1
    file = np.zeros(sum(fileEncodee), dtype = int)
    i = 0
    for v in fileEncodee :
        while v > 0 :
            file[i] = d
            i += 1
            v -= 1
        d -= 1
    
    return file

In [5]:
decode(fileEncodee)

array([5, 5, 3, 2, 2, 1, 0])

### Encodage d'une station

Une station comportant $f$ files peut être représentée par une matrice avec $f$ éléments, chaque élément listant les états des tenders dans une file. On complète les files par des -1 pour les ramener toutes à la même longueur.

In [6]:
def encodeStation(station, d) :
    # Prend une station et renvoie la matrice comportant la valeur encodée de chaque ligne
    
    if type(station) == np.ndarray :
        station = station.tolist()
    
    stationEncodee = []
    for indexFile in range(len(station)) :
        file = station[indexFile]
        try :
            finFile = file.index(-1)
        except ValueError :
            finFile = len(file)
        stationEncodee.append(encode(file[:finFile], d))
    
    return np.array(stationEncodee, dtype = int)

In [7]:
station = [[5, 5, 3, 2, 2, 1, 0], [3, 3]]
stationEncodee = encodeStation(station, 6)
stationEncodee

array([[0, 2, 0, 1, 2, 1, 1],
       [0, 0, 0, 2, 0, 0, 0]])

In [8]:
def padStation(station) :
    # Prend une station représentée sous forme de liste et la renvoie sous forme de np.array, complété de -1
    
    if type(station) == list :
        longueurFiles = max([len(file) for file in station])
        for indexFile, file in enumerate(station) :
            station[indexFile] = file + [-1] * (longueurFiles - len(file))
    
    return np.array(station)

In [9]:
padStation(station)

array([[ 5,  5,  3,  2,  2,  1,  0],
       [ 3,  3, -1, -1, -1, -1, -1]])

In [10]:
def decodeStation(stationEncodee) :
    # Prend une station encodée et renvoie la liste de ses files décodées
    
    if type(stationEncodee) == list :
        stationEncodee = np.array(stationEncodee)
    
    station = []
    
    for indexFile in range(stationEncodee.shape[0]) :
        fileEncodee = stationEncodee[indexFile, :]
        station.append(decode(fileEncodee).tolist())
        
    return padStation(station)

In [11]:
decodeStation(stationEncodee)

array([[ 5,  5,  3,  2,  2,  1,  0],
       [ 3,  3, -1, -1, -1, -1, -1]])

## Charge des tenders

Une même fonction permet de réaliser la charge des stations et des files. Il faut néanmoins différencier le cas encodé et décodé.

In [12]:
def charge(station, d) :
    # Prend une station (décodée) et la renvoie après que chaque tender ait été chargé de 1
    
    if type(station) == list :
        station = padStation(station)
        
    station += ~np.isin(station, [-1, d])    
    
    return station

In [13]:
station

[[5, 5, 3, 2, 2, 1, 0], [3, 3, -1, -1, -1, -1, -1]]

In [14]:
charge(station, 5)

array([[ 5,  5,  4,  3,  3,  2,  1],
       [ 4,  4, -1, -1, -1, -1, -1]])

In [15]:
def chargeEncodee(station) :
    # Prend une station (encodée) et la renvoie après que chaque tender ait été chargé de 1
    
    station = padStation(station)    
    chargedStation = np.zeros(station.shape)
    
    for indexFile in range(station.shape[0]) :
        chargedStation[indexFile, 0] = station[indexFile, 0]
        chargedStation[indexFile, :-1] += station[indexFile, 1:]
    
    return chargedStation

In [16]:
stationEncodee

array([[0, 2, 0, 1, 2, 1, 1],
       [0, 0, 0, 2, 0, 0, 0]])

In [17]:
chargeEncodee(stationEncodee)

array([[2., 0., 1., 2., 1., 1., 0.],
       [0., 0., 2., 0., 0., 0., 0.]])

In [18]:
chargeEncodee(chargeEncodee(chargeEncodee(stationEncodee)))

array([[3., 2., 1., 1., 0., 0., 0.],
       [2., 0., 0., 0., 0., 0., 0.]])

## Réduction du nombre de configurations pour les stations

Les états de notre chaîne de Markov sont les configurations de la station. D'un point de vue de la satisfaction de la demande, cependant, certaines configurations sont équivalentes : on peut permuter librement les files. Ainsi, on ne s'intéresse qu'aux configurations où le nombre de tenders par file est décroissant, et les files avec le même nombre de tenders sont ordonnées selon l'ordre lexicographique décroissant. Remarquons que l'ordre lexicographique est le même qu'on soit en version encodée ou décodée, en revanche la contrainte imposée par le nombre de tenders dans chaque file s'exprime différemment selon que la station est encodée ou non.

### Parcours d'ensembles de suite dans l'ordre lexicographique décroissant

Dans la suite, nous allons avoir besoin de fonctions élémentaires permettant de parcourir, dans l'ordre lexicographique décroissant, l'ensemble des suites vérifiant des contraintes données.

In [19]:
def suiteSuivanteDecSumCst(l) :
    # Permet de parcourir l'ensemble des suites décroissantes de longueur et somme données, selon l'ordre lexicographique
    # décroissant. Renvoie False quand on a atteint le dernier état.
    
    l = l.copy()
    i = len(l) - 1
    curSum = 1
    
    while i >= 0 and curSum > (l[i] - 1) * (len(l) - 1 - i):
        curSum += l[i]
        i -= 1
    
    if i >= 0 :
        l[i] -= 1
        j = i + 1
        while curSum > 0 :
            l[j] = min([curSum, l[i]])
            curSum -= l[j]
            j += 1
        while j < len(l) :
            l[j] = 0
            j += 1
    else :
        l = False
    
    return l    

In [20]:
l = [5, 4, 2, 0]
while type(l) != bool :
    print(l)
    l = suiteSuivanteDecSumCst(l)

[5, 4, 2, 0]
[5, 4, 1, 1]
[5, 3, 3, 0]
[5, 3, 2, 1]
[5, 2, 2, 2]
[4, 4, 3, 0]
[4, 4, 2, 1]
[4, 3, 3, 1]
[4, 3, 2, 2]
[3, 3, 3, 2]


In [21]:
def suiteSuivanteSumCst(l) :
    # Permet de parcourir l'ensemble des suites de longueur et somme données, selon l'ordre lexicographique
    # décroissant. Renvoie False quand on a atteint le dernier état.
    
    l = l.copy()
    
    if len(l) >= 2 :
        i = len(l) - 2
        curSum = l[i + 1]
    
        while i >= 0 :
            if l[i] >= 1 :
                l[i] -= 1
                l[i + 1] = curSum + 1
                l[(i + 2):] = [0] * (len(l) - 2 - i)
                break
            else :
                curSum += l[i]
                i -= 1
        
        if i == -1 :
            l = False
            
    else :
        l = False
    
    return l

In [22]:
l = [3, 0, 0]
while type(l) != bool :
    print(l)
    l = suiteSuivanteSumCst(l)

[3, 0, 0]
[2, 1, 0]
[2, 0, 1]
[1, 2, 0]
[1, 1, 1]
[1, 0, 2]
[0, 3, 0]
[0, 2, 1]
[0, 1, 2]
[0, 0, 3]


In [23]:
def suiteSuivanteDec(l) :
    # Permet de parcourir l'ensemble des suites décroissantes de longueur donnée, selon l'ordre lexicographique
    # décroissant. Renvoie False quand on a atteint le dernier état. On spécifie la valeur maximale en commençant par
    # d, d, d, ..., d
    
    l = l.copy()
    i = len(l) - 1
    
    while i >= 0 :
        if l[i] >= 1 :
            l[i] -= 1
            l[(i+1):] = [l[i]] * (len(l) - 1 - i)
            break
        i -= 1
        
    if i == -1 :
        l = False
    
    return l

In [24]:
l = [3, 3, 3]

while type(l) != bool :
    print(l)
    l = suiteSuivanteDec(l)

[3, 3, 3]
[3, 3, 2]
[3, 3, 1]
[3, 3, 0]
[3, 2, 2]
[3, 2, 1]
[3, 2, 0]
[3, 1, 1]
[3, 1, 0]
[3, 0, 0]
[2, 2, 2]
[2, 2, 1]
[2, 2, 0]
[2, 1, 1]
[2, 1, 0]
[2, 0, 0]
[1, 1, 1]
[1, 1, 0]
[1, 0, 0]
[0, 0, 0]


### Parcours de l'ensemble des configurations possibles pour une station

Pour les stations encodées, on a :

In [40]:
def etatEncodeSuivant(etat) :
    # Permet de parcourir l'ensemble des états possibles pour une station avec un nombre donné de tenders déjà
    # répartis entre les files, sans répéter deux états équivalents. Parcourt la répartition dans chaque file
    # dans l'ordre lexicographique décroissant. Les stations sont sous forme encodée.
    
    etat = etat.copy()
    
    nFile = etat.shape[0]
    curFile = nFile - 1
    
    while curFile >= 0 :
        file = etat[curFile, :]
        
        if type(suiteSuivanteSumCst(file)) == bool :
            file[0] = sum(file)
            file[1:] = 0
            etat[curFile, :] = file
        else :
            file = suiteSuivanteSumCst(file)
            etat[curFile, :] = file
            
            # On s'assure que les files avec le même nombre de tenders sont bien plus petites selon l'ordre lexicographique
            j = curFile + 1
            nbTenders = sum(file)
            while j < nFile and etat[j, 0] == nbTenders :
                etat[j, :] = file
            
            break

        curFile -= 1
    
    if curFile == -1 :
        etat = False
        
    return etat

In [41]:
def stationEncodeeSuivante(station) :
    # Permet de parcourir l'ensemble des états possibles pour une station avec un nombre donné de tenders,
    # sans répéter deux états équivalents. Parcourt la répartition des tenders par file selon l'ordre lexicographique
    # décroissant, puis à répartition donnée chaque file dans l'ordre lexicographique décroissant. Les stations sont sous
    # forme encodée.
    
    if type(etatEncodeSuivant(station)) != bool :
        station = etatEncodeSuivant(station)
    else :
        tenderParFile = np.sum(station, axis = 1)
        if type(suiteSuivanteDecSumCst(tenderParFile)) == bool :
            station = False
        else :
            tenderParFile = suiteSuivanteDecSumCst(tenderParFile)
            station[:, :] = 0
            station[:, 0] = tenderParFile
    
    return station

In [42]:
etat = padStation([[4, 0], [0, 0]])

while type(etat) != bool :
    print(etat)
    etat = stationEncodeeSuivante(etat)

[[4 0]
 [0 0]]
[[3 1]
 [0 0]]
[[2 2]
 [0 0]]
[[1 3]
 [0 0]]
[[0 4]
 [0 0]]
[[3 0]
 [1 0]]
[[3 0]
 [0 1]]
[[2 1]
 [1 0]]
[[2 1]
 [0 1]]
[[1 2]
 [1 0]]
[[1 2]
 [0 1]]
[[0 3]
 [1 0]]
[[0 3]
 [0 1]]
[[2 0]
 [2 0]]
[[2 0]
 [1 1]]
[[2 0]
 [0 2]]
[[1 1]
 [1 1]]
[[1 1]
 [0 2]]
[[0 2]
 [0 2]]


Pour les stations décodées, on a :

In [ ]:
def etatDecodeSuivant(etat, d) :
    # Permet de parcourir l'ensemble des états possibles pour une station avec un nombre donné de tenders déjà
    # répartis entre les files, sans répéter deux états équivalents. Parcourt la répartition dans chaque file
    # dans l'ordre lexicographique décroissant. Les stations sont sous forme décodée, avec des états de charge de 0 à d.
    
    etat = etat.copy()
    
    nFile = etat.shape[0]
    curFile = nFile - 1
    
    while curFile >= 0 :
        
        file = etat[curFile, :]
        try :
            finFile = file.index(-1)
        except ValueError :
            finFile = len(file)
            
        if finFile >= 1 :
            file = file[:finFile]

            if type(suiteSuivanteDec(file)) == bool :
                file[:] = d
            else :
                file = suiteSuivanteDec(file)
                etat[curFile, :finFile] = file

                # On s'assure que les files avec le même nombre de tenders sont bien plus petites selon l'ordre lexicographique
                j = curFile + 1
                while j < nFile and etat[j, finFile - 1] == :
                    ### A SUIVRE

                break

        curFile -= 1
    
    if curFile == -1 :
        etat = False
        
    return etat

In [43]:
def stationDecodeeSuivante(station) :
    # Permet de parcourir l'ensemble des états possibles pour une station avec un nombre donné de tenders,
    # sans répéter deux états équivalents. Parcourt la répartition des tenders par file selon l'ordre lexicographique
    # décroissant, puis à répartition donnée chaque file dans l'ordre lexicographique décroissant. Les stations sont sous
    # forme encodée.
    
    if type(etatEncodeSuivant(station)) != bool :
        station = etatEncodeSuivant(station)
    else :
        tenderParFile = np.sum(station, axis = 1)
        if type(suiteSuivanteDecSumCst(tenderParFile)) == bool :
            station = False
        else :
            tenderParFile = suiteSuivanteDecSumCst(tenderParFile)
            station[:, :] = 0
            station[:, 0] = tenderParFile
    
    return station